In [3]:
!pip install snowflake

In [ ]:
import os

In [4]:
!pip install snowflake-connector-python

In [11]:
import os 
import snowflake.connector

# Retrieve credentials from environment variables
user=os.getenv('SNOWFLAKE_USER','ELI')
password=os.getenv('SNOWFLAKE_PASSWORD')
account=os.getenv('SNOWFLAKE_ACCOUNT','bcqgjlk-pm69037')

# Prompt for password if not set in environment
if not password:
    password=input("Enter your Snowflake password: ")


Enter your Snowflake password: Elhamyadollahi123


In [ ]:
#commond prompt
#set SNOWFLAKE_USER=Eli
#set SNOWFLAKE_PASSWORD=YourPassword
#set SNOWFLAKE_ACCOUNT=bcqgjlk.pm69037
#echo %SNOWFLAKE_USER%


In [12]:
conn = snowflake.connector.connect(
        user='Eli',
        password=password,
        #account='PM69037',
        account='bcqgjlk-pm69037',
        warehouse='NEW',
        database='learn',
        schema='learn_schema',
        role='ACCOUNTADMIN'
    )
print("Connected to Snowflake successfully!")



Connected to Snowflake successfully!


In [18]:
# Create a cursor
cursor=conn.cursor()

# Ensure the stage exists
cursor.execute("""
CREATE STAGE IF NOT EXISTS my_stage2
        COMMENT = 'Stage for loading customer CSV files'
""")

print("Stage 'my_stage' created or verified.")

Stage 'my_stage' created or verified.


In [19]:
 # Grant permissions to ensure ACCOUNTADMIN can use the stage
cursor.execute("""
GRANT READ,WRITE ON STAGE my_stage2 TO ROLE ACCOUNTADMIN
        
    """)
print("Stage permissions granted.")


Stage permissions granted.


In [27]:
# Path to the CSV file (using raw string for Windows paths)
CSV_FILE_PATH = r'C:\Users\elham\OneDrive\Desktop\applic\SNHOWFLAKE\new.csv'

# Convert backslashes to forward slashes for Snowflake PUT command
PUT_FILE_PATH = CSV_FILE_PATH.replace('\\', '/')

In [28]:
 cursor.execute(f"""
 PUT file://{PUT_FILE_PATH} @my_stage2
 AUTO_COMPRESS=TRUE
            
            """)

In [29]:
 # List files in the stage
cursor.execute("LIST @my_stage2")

In [30]:
 # List files in the stage
cursor.execute("LIST @my_stage2")
files = cursor.fetchall()
print("Files in @my_stage2:")
for file in files:
    print(file)


Files in @my_stage2:
('my_stage2/new.csv.gz', 208, '854a1fa4da4005740ea32b3685f20837', 'Mon, 5 May 2025 15:35:04 GMT')


In [32]:

 # Load data into the customers table
cursor.execute("""
        COPY INTO customer
        FROM @my_stage2/new.csv
        FILE_FORMAT = (
            TYPE = CSV
            SKIP_HEADER = 1
            FIELD_OPTIONALLY_ENCLOSED_BY = '"'
            DATE_FORMAT = 'YYYY-MM-DD'
        )
        ON_ERROR = 'CONTINUE'
    """)
print("Data loaded into customers table.")

Data loaded into customers table.


In [33]:
 # Verify the loaded data
cursor.execute("SELECT * FROM customer")
rows = cursor.fetchall()
print("Data in customers table:")
for row in rows:
    print(row)


Data in customers table:
(3, 'Charlie Lee', 'Asia', 15000.0, datetime.date(2024, 9, 10))
(1, 'Alice Smith', 'North America', 13200.550000000001, datetime.date(2024, 3, 15))
(4, 'Dana Kim', 'North America', 10450.275000000001, datetime.date(2025, 1, 5))
(5, 'Eve Brown', 'Europe', 11000.0, datetime.date(2025, 2, 1))
(6, 'Frank Wilson', 'Asia', 13000.75, datetime.date(2025, 3, 12))
(7, 'Grace Chen', 'North America', 9800.5, datetime.date(2025, 4, 20))
(8, 'Henry Patel', 'Australia', 14500.25, datetime.date(2025, 5, 1))


In [34]:
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.
